This notebook consist of the process to retreive data of three seperate datasets:
- All-nba teams
- All-stars
- All-stars 2020

In [1]:
import pandas as pd
import sys 

sys.path.insert(1, '../src/retrieve_data')

pd.set_option('display.max_rows', 800)
pd.set_option('display.max_columns', 800)

## Part 1: All-nba teams

First, scrape data to get players who were on the all-nba team. Notice that, the All-NBA team has been composed of three teams since the 1988-89 season. Data is fetched from 1988-1989 season to 2018-2019 season.

In [2]:
import all_nba_team_scraper

In [3]:
all_nba_teams = all_nba_team_scraper.all_nba_team()

In [5]:
all_nba_teams.to_csv('..//data//All_nba_team_csv', index = False) # saving the csv file

In [6]:
all_nba_teams['All nba team'].value_counts()

1st    155
3rd    155
2nd    155
Name: All nba team, dtype: int64

## Part 2: All-star players
Second, scrape data to get players who were all-stars. Since we fetched data for players who were on the All-NBA team since the the 1988-89 season, we will fetch follow this format and retreive data of players who were all-stars between 1988-1989 season to 2018-2019 season

In [7]:
dfs = []
for year in range(1989,2020,1):
    if year == 1999: # Due to lockout, there was no all stars in 1999
        continue
        
    df = all_nba_team_scraper.all_stars(year)
    dfs.append(df)
    
star_players = pd.concat(dfs).reset_index(drop = True)

In [8]:
star_players.head()

,Players,Season
0,John Stockton,1989
1,Alex English,1989
2,Karl Malone,1989
3,Dale Ellis,1989
4,Hakeem Olajuwon,1989


In [9]:
star_players['Players'] = star_players['Players'].apply(all_nba_team_scraper.clean_text)

In the 2018-2019 season, Dwayne Wade and Dirk Nowitzki were named as special team roster additions for the game, to commemorate their contribution to the game of basektball. From an avid basketball perspective, they are outliers and would not be chosen as all-stars based on votings and performance. Therefore, we will remove them in our dataset

In [10]:
star_players.loc[(star_players['Players'] == 'Dwyane Wade') & (star_players['Season'] == 2019)]

,Players,Season
706,Dwyane Wade,2019


In [11]:
star_players.loc[(star_players['Players'] == 'Dirk Nowitzki') & (star_players['Season'] == 2019)]

,Players,Season
720,Dirk Nowitzki,2019


In [12]:
star_players = star_players.drop([706,720])

In [13]:
star_players = star_players.reset_index(drop = True)

## Part 3: Player stats 

Thirdly, scrape data to get players stats. Again, we are trying to retreieve data starting from 1989-1989 season. 

In [14]:
import nba_stats

In [15]:
result = nba_stats.player_stats_seasons(1989,2019)

In [16]:
result

,Player,Pos,Age,Tm,G,GS,MP_x,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,MP_y,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,,OWS,DWS,WS,WS/48,,OBPM,DBPM,BPM,VORP,Season
0,Kareem Abdul-Jabbar*,C,41,LAL,74,74,22.9,4.2,8.9,.475,0.0,0.0,.000,4.2,8.9,.477,.475,1.6,2.2,.739,1.4,3.1,4.5,1.0,0.5,1.1,1.3,2.6,10.1,1695,12.9,.511,.005,.250,7.3,14.2,11.0,6.1,1.1,2.9,11.5,20.1,,0.9,2.0,2.9,.082,,-1.9,-0.2,-2.1,0.0,1989
1,Mark Acres,C,26,BOS,62,0,10.2,0.9,1.8,.482,0.0,0.0,1.000,0.9,1.8,.478,.487,0.4,0.8,.542,1.0,1.4,2.4,0.3,0.3,0.1,0.4,1.5,2.2,632,8.2,.507,.009,.421,10.9,15.6,13.3,3.7,1.5,0.6,14.5,10.4,,0.4,0.5,0.9,.066,,-3.0,-0.4,-3.4,-0.2,1989
2,Michael Adams,PG,26,DEN,77,77,36.2,6.1,14.1,.433,2.2,6.1,.356,3.9,8.0,.490,.509,4.2,5.1,.819,0.9,2.8,3.7,6.4,2.2,0.1,2.3,1.9,18.5,2787,17.5,.567,.431,.363,2.5,8.3,5.2,22.2,2.7,0.2,12.5,19.7,,5.4,2.9,8.3,.143,,2.7,0.2,2.9,3.4,1989
3,Mark Aguirre,SF,29,TOT,80,76,32.5,7.3,15.9,.461,0.6,2.2,.293,6.7,13.7,.488,.481,3.6,4.9,.733,1.8,3.0,4.8,3.5,0.6,0.5,2.6,2.9,18.9,2597,16.4,.524,.137,.309,6.4,10.0,8.3,17.9,0.9,0.8,12.6,27.4,,2.4,2.1,4.5,.082,,1.4,-1.3,0.1,1.4,1989
4,Mark Aguirre,SF,29,DAL,44,44,34.8,8.5,18.8,.450,0.7,2.3,.293,7.8,16.6,.471,.467,4.0,5.5,.730,2.0,3.3,5.3,4.3,0.7,0.7,3.2,2.9,21.7,1529,17.3,.509,.119,.294,6.6,10.3,8.5,21.5,0.9,1.1,13.0,30.5,,0.7,1.0,1.7,.052,,1.5,-1.9,-0.4,0.6,1989
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
703,Tyler Zeller,C,29,MEM,4,1,20.5,4.0,7.0,.571,0.0,0.0,,4.0,7.0,.571,.571,3.5,4.5,.778,2.3,2.3,4.5,0.8,0.3,0.8,1.0,4.0,11.5,82,19.4,.640,.000,.643,12.1,12.5,12.3,6.2,0.6,3.5,10.0,21.8,,0.2,0.1,0.3,.191,,-2.3,-1.9,-4.3,0.0,2019
704,Ante Žižić,C,22,CLE,59,25,18.3,3.1,5.6,.553,0.0,0.0,,3.1,5.6,.553,.553,1.6,2.2,.705,1.8,3.6,5.4,0.9,0.2,0.4,1.0,1.9,7.8,1082,16.2,.590,.000,.399,10.6,22.8,16.4,7.7,0.6,1.8,13.6,18.2,,1.7,0.3,2.0,.087,,-1.1,-2.1,-3.2,-0.3,2019
705,Ivica Zubac,C,21,TOT,59,37,17.6,3.6,6.4,.559,0.0,0.0,,3.6,6.4,.559,.559,1.7,2.1,.802,1.9,4.2,6.1,1.1,0.2,0.9,1.2,2.3,8.9,1040,18.9,.604,.000,.332,11.9,24.2,18.2,9.1,0.6,4.0,13.9,20.3,,1.9,1.3,3.2,.148,,-0.2,-0.2,-0.5,0.4,2019
706,Ivica Zubac,C,21,LAL,33,12,15.6,3.4,5.8,.580,0.0,0.0,,3.4,5.8,.580,.580,1.7,2.0,.864,1.6,3.3,4.9,0.8,0.1,0.8,1.0,2.2,8.5,516,19.5,.633,.000,.342,11.0,21.2,16.2,7.3,0.4,4.3,12.9,20.5,,1.1,0.6,1.8,.165,,-0.3,0.1,-0.2,0.2,2019


Similar to when retrieving data for all nba team, there are a few cases where a player changed his name. Luckily the data that has been retrieved already has names taken account of and changed.

Clean up the players names

In [17]:
result['Player'] = result['Player'].apply(all_nba_team_scraper.clean_text)

In [18]:
result = result.reset_index(drop = True)

In [19]:
result

,Player,Pos,Age,Tm,G,GS,MP_x,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,MP_y,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,,OWS,DWS,WS,WS/48,,OBPM,DBPM,BPM,VORP,Season
0,Kareem Abdul-Jabbar,C,41,LAL,74,74,22.9,4.2,8.9,.475,0.0,0.0,.000,4.2,8.9,.477,.475,1.6,2.2,.739,1.4,3.1,4.5,1.0,0.5,1.1,1.3,2.6,10.1,1695,12.9,.511,.005,.250,7.3,14.2,11.0,6.1,1.1,2.9,11.5,20.1,,0.9,2.0,2.9,.082,,-1.9,-0.2,-2.1,0.0,1989
1,Mark Acres,C,26,BOS,62,0,10.2,0.9,1.8,.482,0.0,0.0,1.000,0.9,1.8,.478,.487,0.4,0.8,.542,1.0,1.4,2.4,0.3,0.3,0.1,0.4,1.5,2.2,632,8.2,.507,.009,.421,10.9,15.6,13.3,3.7,1.5,0.6,14.5,10.4,,0.4,0.5,0.9,.066,,-3.0,-0.4,-3.4,-0.2,1989
2,Michael Adams,PG,26,DEN,77,77,36.2,6.1,14.1,.433,2.2,6.1,.356,3.9,8.0,.490,.509,4.2,5.1,.819,0.9,2.8,3.7,6.4,2.2,0.1,2.3,1.9,18.5,2787,17.5,.567,.431,.363,2.5,8.3,5.2,22.2,2.7,0.2,12.5,19.7,,5.4,2.9,8.3,.143,,2.7,0.2,2.9,3.4,1989
3,Mark Aguirre,SF,29,TOT,80,76,32.5,7.3,15.9,.461,0.6,2.2,.293,6.7,13.7,.488,.481,3.6,4.9,.733,1.8,3.0,4.8,3.5,0.6,0.5,2.6,2.9,18.9,2597,16.4,.524,.137,.309,6.4,10.0,8.3,17.9,0.9,0.8,12.6,27.4,,2.4,2.1,4.5,.082,,1.4,-1.3,0.1,1.4,1989
4,Mark Aguirre,SF,29,DAL,44,44,34.8,8.5,18.8,.450,0.7,2.3,.293,7.8,16.6,.471,.467,4.0,5.5,.730,2.0,3.3,5.3,4.3,0.7,0.7,3.2,2.9,21.7,1529,17.3,.509,.119,.294,6.6,10.3,8.5,21.5,0.9,1.1,13.0,30.5,,0.7,1.0,1.7,.052,,1.5,-1.9,-0.4,0.6,1989
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16922,Tyler Zeller,C,29,MEM,4,1,20.5,4.0,7.0,.571,0.0,0.0,,4.0,7.0,.571,.571,3.5,4.5,.778,2.3,2.3,4.5,0.8,0.3,0.8,1.0,4.0,11.5,82,19.4,.640,.000,.643,12.1,12.5,12.3,6.2,0.6,3.5,10.0,21.8,,0.2,0.1,0.3,.191,,-2.3,-1.9,-4.3,0.0,2019
16923,Ante Žižić,C,22,CLE,59,25,18.3,3.1,5.6,.553,0.0,0.0,,3.1,5.6,.553,.553,1.6,2.2,.705,1.8,3.6,5.4,0.9,0.2,0.4,1.0,1.9,7.8,1082,16.2,.590,.000,.399,10.6,22.8,16.4,7.7,0.6,1.8,13.6,18.2,,1.7,0.3,2.0,.087,,-1.1,-2.1,-3.2,-0.3,2019
16924,Ivica Zubac,C,21,TOT,59,37,17.6,3.6,6.4,.559,0.0,0.0,,3.6,6.4,.559,.559,1.7,2.1,.802,1.9,4.2,6.1,1.1,0.2,0.9,1.2,2.3,8.9,1040,18.9,.604,.000,.332,11.9,24.2,18.2,9.1,0.6,4.0,13.9,20.3,,1.9,1.3,3.2,.148,,-0.2,-0.2,-0.5,0.4,2019
16925,Ivica Zubac,C,21,LAL,33,12,15.6,3.4,5.8,.580,0.0,0.0,,3.4,5.8,.580,.580,1.7,2.0,.864,1.6,3.3,4.9,0.8,0.1,0.8,1.0,2.2,8.5,516,19.5,.633,.000,.342,11.0,21.2,16.2,7.3,0.4,4.3,12.9,20.5,,1.1,0.6,1.8,.165,,-0.3,0.1,-0.2,0.2,2019


Create a dictionary of years abbreviated and write the full year

In [20]:
mydict = {}

for i in range(89,100,1):
    mydict[str(i)] = '19{}'.format(i)
    
for i in range(0,21,1):
    
    if len(str(i)) == 1:
        mydict[('%0.2d' % i)] = '200{}'.format(i)
    else:   
        mydict[str(i)] = '20{}'.format(i)

In [21]:
all_nba_teams['Season_end'] = all_nba_teams['Season'].str[-2:] # Create a new column

In [22]:
all_nba_teams = all_nba_teams.replace({'Season_end': mydict}) # replace with dictionary values
all_nba_teams['Season_end'] = all_nba_teams['Season_end'].astype(int)

Merge player stats df with all-nba teams df

In [23]:
nba_df = result.merge(all_nba_teams, how= 'left', left_on = ['Player', 'Season'], right_on = ['Players', 'Season_end'])

nba_df = nba_df.rename(columns ={'All nba team': 'NBA_ALL_TEAM'})

In [24]:
nba_df.head(5)

,Player,Pos,Age,Tm,G,GS,MP_x,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,MP_y,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,,OWS,DWS,WS,WS/48,,OBPM,DBPM,BPM,VORP,Season_x,Players,Season_y,NBA_ALL_TEAM,Season_end
0,Kareem Abdul-Jabbar,C,41,LAL,74,74,22.9,4.2,8.9,.475,0.0,0.0,.000,4.2,8.9,.477,.475,1.6,2.2,.739,1.4,3.1,4.5,1.0,0.5,1.1,1.3,2.6,10.1,1695,12.9,.511,.005,.250,7.3,14.2,11.0,6.1,1.1,2.9,11.5,20.1,,0.9,2.0,2.9,.082,,-1.9,-0.2,-2.1,0.0,1989,NaN,NaN,NaN,NaN
1,Mark Acres,C,26,BOS,62,0,10.2,0.9,1.8,.482,0.0,0.0,1.000,0.9,1.8,.478,.487,0.4,0.8,.542,1.0,1.4,2.4,0.3,0.3,0.1,0.4,1.5,2.2,632,8.2,.507,.009,.421,10.9,15.6,13.3,3.7,1.5,0.6,14.5,10.4,,0.4,0.5,0.9,.066,,-3.0,-0.4,-3.4,-0.2,1989,NaN,NaN,NaN,NaN
2,Michael Adams,PG,26,DEN,77,77,36.2,6.1,14.1,.433,2.2,6.1,.356,3.9,8.0,.490,.509,4.2,5.1,.819,0.9,2.8,3.7,6.4,2.2,0.1,2.3,1.9,18.5,2787,17.5,.567,.431,.363,2.5,8.3,5.2,22.2,2.7,0.2,12.5,19.7,,5.4,2.9,8.3,.143,,2.7,0.2,2.9,3.4,1989,NaN,NaN,NaN,NaN
3,Mark Aguirre,SF,29,TOT,80,76,32.5,7.3,15.9,.461,0.6,2.2,.293,6.7,13.7,.488,.481,3.6,4.9,.733,1.8,3.0,4.8,3.5,0.6,0.5,2.6,2.9,18.9,2597,16.4,.524,.137,.309,6.4,10.0,8.3,17.9,0.9,0.8,12.6,27.4,,2.4,2.1,4.5,.082,,1.4,-1.3,0.1,1.4,1989,NaN,NaN,NaN,NaN
4,Mark Aguirre,SF,29,DAL,44,44,34.8,8.5,18.8,.450,0.7,2.3,.293,7.8,16.6,.471,.467,4.0,5.5,.730,2.0,3.3,5.3,4.3,0.7,0.7,3.2,2.9,21.7,1529,17.3,.509,.119,.294,6.6,10.3,8.5,21.5,0.9,1.1,13.0,30.5,,0.7,1.0,1.7,.052,,1.5,-1.9,-0.4,0.6,1989,NaN,NaN,NaN,NaN


In [25]:
star_players['ALL_STAR'] = 'YES' # Column 'YES' if they are all-star

Merge the results from above with all-stars df

In [26]:
final_df = nba_df.merge(star_players, how = 'left', left_on = ['Players', 'Season_x'], right_on = ['Players', 'Season'])

In [27]:
final_df.head()

,Player,Pos,Age,Tm,G,GS,MP_x,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,MP_y,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,,OWS,DWS,WS,WS/48,,OBPM,DBPM,BPM,VORP,Season_x,Players,Season_y,NBA_ALL_TEAM,Season_end,Season,ALL_STAR
0,Kareem Abdul-Jabbar,C,41,LAL,74,74,22.9,4.2,8.9,.475,0.0,0.0,.000,4.2,8.9,.477,.475,1.6,2.2,.739,1.4,3.1,4.5,1.0,0.5,1.1,1.3,2.6,10.1,1695,12.9,.511,.005,.250,7.3,14.2,11.0,6.1,1.1,2.9,11.5,20.1,,0.9,2.0,2.9,.082,,-1.9,-0.2,-2.1,0.0,1989,NaN,NaN,NaN,NaN,NaN,NaN
1,Mark Acres,C,26,BOS,62,0,10.2,0.9,1.8,.482,0.0,0.0,1.000,0.9,1.8,.478,.487,0.4,0.8,.542,1.0,1.4,2.4,0.3,0.3,0.1,0.4,1.5,2.2,632,8.2,.507,.009,.421,10.9,15.6,13.3,3.7,1.5,0.6,14.5,10.4,,0.4,0.5,0.9,.066,,-3.0,-0.4,-3.4,-0.2,1989,NaN,NaN,NaN,NaN,NaN,NaN
2,Michael Adams,PG,26,DEN,77,77,36.2,6.1,14.1,.433,2.2,6.1,.356,3.9,8.0,.490,.509,4.2,5.1,.819,0.9,2.8,3.7,6.4,2.2,0.1,2.3,1.9,18.5,2787,17.5,.567,.431,.363,2.5,8.3,5.2,22.2,2.7,0.2,12.5,19.7,,5.4,2.9,8.3,.143,,2.7,0.2,2.9,3.4,1989,NaN,NaN,NaN,NaN,NaN,NaN
3,Mark Aguirre,SF,29,TOT,80,76,32.5,7.3,15.9,.461,0.6,2.2,.293,6.7,13.7,.488,.481,3.6,4.9,.733,1.8,3.0,4.8,3.5,0.6,0.5,2.6,2.9,18.9,2597,16.4,.524,.137,.309,6.4,10.0,8.3,17.9,0.9,0.8,12.6,27.4,,2.4,2.1,4.5,.082,,1.4,-1.3,0.1,1.4,1989,NaN,NaN,NaN,NaN,NaN,NaN
4,Mark Aguirre,SF,29,DAL,44,44,34.8,8.5,18.8,.450,0.7,2.3,.293,7.8,16.6,.471,.467,4.0,5.5,.730,2.0,3.3,5.3,4.3,0.7,0.7,3.2,2.9,21.7,1529,17.3,.509,.119,.294,6.6,10.3,8.5,21.5,0.9,1.1,13.0,30.5,,0.7,1.0,1.7,.052,,1.5,-1.9,-0.4,0.6,1989,NaN,NaN,NaN,NaN,NaN,NaN


## Clean up the column names

In [28]:
final_df = final_df.rename(columns = {'MP_y': 'MP',
                                      'MP_x': 'MP/G'})

In [29]:
cols = [col for col in final_df.columns if col not in ['Players', 'Season_y', 'Season_end', 'Season']]

In [30]:
final_df = final_df[cols]

In [31]:
final_df

,Player,Pos,Age,Tm,G,GS,MP/G,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,,,OWS,DWS,WS,WS/48,,,OBPM,DBPM,BPM,VORP,Season_x,NBA_ALL_TEAM,ALL_STAR
0,Kareem Abdul-Jabbar,C,41,LAL,74,74,22.9,4.2,8.9,.475,0.0,0.0,.000,4.2,8.9,.477,.475,1.6,2.2,.739,1.4,3.1,4.5,1.0,0.5,1.1,1.3,2.6,10.1,1695,12.9,.511,.005,.250,7.3,14.2,11.0,6.1,1.1,2.9,11.5,20.1,,,0.9,2.0,2.9,.082,,,-1.9,-0.2,-2.1,0.0,1989,NaN,NaN
1,Mark Acres,C,26,BOS,62,0,10.2,0.9,1.8,.482,0.0,0.0,1.000,0.9,1.8,.478,.487,0.4,0.8,.542,1.0,1.4,2.4,0.3,0.3,0.1,0.4,1.5,2.2,632,8.2,.507,.009,.421,10.9,15.6,13.3,3.7,1.5,0.6,14.5,10.4,,,0.4,0.5,0.9,.066,,,-3.0,-0.4,-3.4,-0.2,1989,NaN,NaN
2,Michael Adams,PG,26,DEN,77,77,36.2,6.1,14.1,.433,2.2,6.1,.356,3.9,8.0,.490,.509,4.2,5.1,.819,0.9,2.8,3.7,6.4,2.2,0.1,2.3,1.9,18.5,2787,17.5,.567,.431,.363,2.5,8.3,5.2,22.2,2.7,0.2,12.5,19.7,,,5.4,2.9,8.3,.143,,,2.7,0.2,2.9,3.4,1989,NaN,NaN
3,Mark Aguirre,SF,29,TOT,80,76,32.5,7.3,15.9,.461,0.6,2.2,.293,6.7,13.7,.488,.481,3.6,4.9,.733,1.8,3.0,4.8,3.5,0.6,0.5,2.6,2.9,18.9,2597,16.4,.524,.137,.309,6.4,10.0,8.3,17.9,0.9,0.8,12.6,27.4,,,2.4,2.1,4.5,.082,,,1.4,-1.3,0.1,1.4,1989,NaN,NaN
4,Mark Aguirre,SF,29,DAL,44,44,34.8,8.5,18.8,.450,0.7,2.3,.293,7.8,16.6,.471,.467,4.0,5.5,.730,2.0,3.3,5.3,4.3,0.7,0.7,3.2,2.9,21.7,1529,17.3,.509,.119,.294,6.6,10.3,8.5,21.5,0.9,1.1,13.0,30.5,,,0.7,1.0,1.7,.052,,,1.5,-1.9,-0.4,0.6,1989,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16922,Tyler Zeller,C,29,MEM,4,1,20.5,4.0,7.0,.571,0.0,0.0,,4.0,7.0,.571,.571,3.5,4.5,.778,2.3,2.3,4.5,0.8,0.3,0.8,1.0,4.0,11.5,82,19.4,.640,.000,.643,12.1,12.5,12.3,6.2,0.6,3.5,10.0,21.8,,,0.2,0.1,0.3,.191,,,-2.3,-1.9,-4.3,0.0,2019,NaN,NaN
16923,Ante Žižić,C,22,CLE,59,25,18.3,3.1,5.6,.553,0.0,0.0,,3.1,5.6,.553,.553,1.6,2.2,.705,1.8,3.6,5.4,0.9,0.2,0.4,1.0,1.9,7.8,1082,16.2,.590,.000,.399,10.6,22.8,16.4,7.7,0.6,1.8,13.6,18.2,,,1.7,0.3,2.0,.087,,,-1.1,-2.1,-3.2,-0.3,2019,NaN,NaN
16924,Ivica Zubac,C,21,TOT,59,37,17.6,3.6,6.4,.559,0.0,0.0,,3.6,6.4,.559,.559,1.7,2.1,.802,1.9,4.2,6.1,1.1,0.2,0.9,1.2,2.3,8.9,1040,18.9,.604,.000,.332,11.9,24.2,18.2,9.1,0.6,4.0,13.9,20.3,,,1.9,1.3,3.2,.148,,,-0.2,-0.2,-0.5,0.4,2019,NaN,NaN
16925,Ivica Zubac,C,21,LAL,33,12,15.6,3.4,5.8,.580,0.0,0.0,,3.4,5.8,.580,.580,1.7,2.0,.864,1.6,3.3,4.9,0.8,0.1,0.8,1.0,2.2,8.5,516,19.5,.633,.000,.342,11.0,21.2,16.2,7.3,0.4,4.3,12.9,20.5,,,1.1,0.6,1.8,.165,,,-0.3,0.1,-0.2,0.2,2019,NaN,NaN


In [ ]:
final_df['NBA_ALL_TEAM'].value_counts()

In [32]:
# nba_df = test
final_df.to_csv('..//data//nba_df.csv', index = False)

##  Part 4:Retrieve test set  (this season's stats)

In [33]:
import nba_stats

In [34]:
df_per_game = nba_stats.player_stats(2020, 'per_game')
df_advanced = nba_stats.player_stats(2020, 'advanced')

In [35]:
# Determine the matchin column names in these two dataframes
intersect_cols = df_per_game.columns.intersection(df_advanced.columns).tolist()
intersect_cols.remove('MP')

ex = df_per_game.merge(df_advanced, how = 'left', on = intersect_cols)

In [36]:
combined_df = ex.dropna().reset_index(drop = True)

In [37]:
all_star_2020 = all_nba_team_scraper.all_stars(2020)
all_star_2020['ALL_STAR'] = 'YES'

In [38]:
test_set = combined_df.merge(all_star_2020, how = 'left', left_on = 'Player', right_on = 'Players')
test_set['ALL_STAR'] = test_set['ALL_STAR'].fillna('NO')

In [39]:
test_set.to_csv('..//data//nba_19-20.csv',index = False)